In [1]:

import cv2
import numpy as np
import subprocess
import time
import os

def undistort_and_rotate(image_path, camera_matrix, dist_coeffs, angle, output_path):
    start_time = time.time()
    # 读取图像
    image = cv2.imread(image_path)
    
    # 获取新的相机矩阵
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (image.shape[1], image.shape[0]), 1, (image.shape[1], image.shape[0]))
    
    # 畸变矫正
    undistorted_image = cv2.undistort(image, camera_matrix, dist_coeffs, None, new_camera_matrix)
    
    # 裁剪图像
    x, y, w, h = roi
    undistorted_image = undistorted_image[y:y+h, x:x+w]
    
    # 获取图像中心
    (h, w) = undistorted_image.shape[:2]
    center = (w // 2, h // 2)
    
    # 旋转矩阵
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # 旋转图像并裁剪黑边
    rotated_image = cv2.warpAffine(undistorted_image, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
    
    # 保存图像
    cv2.imwrite(output_path, rotated_image)
    end_time = time.time()
    print(f"Saved undistorted and rotated image: {output_path}")
    print(f"Undistort and rotate time: {end_time - start_time:.2f} seconds")

def preprocess_images(image_paths, camera_matrix, dist_coeffs_list, rotation_angles):
    processed_images = []
    for i, image_path in enumerate(image_paths):
        output_path = f'./output/undistorted_image_{i+1}.png'
        undistort_and_rotate(image_path, camera_matrix, dist_coeffs_list[i], rotation_angles[i], output_path)
        processed_images.append(output_path)
    return processed_images

def stitch_images(image1_path, image2_path, output_path):
    start_time = time.time()
    # 构建命令
    command = ["./image_stitching", image1_path, image2_path, output_path]

    # 使用subprocess运行命令
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        end_time = time.time()
        print(f"Stitched images: {image1_path}, {image2_path} to {output_path}")
        print(f"Stitching time: {end_time - start_time:.2f} seconds")
    except subprocess.CalledProcessError as e:
        print("Error occurred while executing the command.")
        print("Error message:\n", e.stderr.decode())
        exit(1)  # 退出程序

def process_image_batches(image_batches, camera_matrix, dist_coeffs_list, rotation_angles):
    if not os.path.exists('./output'):
        os.makedirs('./output')

    for batch_index, image_paths in enumerate(image_batches):
        print(f"Processing batch {batch_index+1}")
        # 预处理图像
        processed_images = preprocess_images(image_paths, camera_matrix, dist_coeffs_list, rotation_angles)
        # 拼接图像
        stitch_images(processed_images[0], processed_images[1], f'./output/output_{batch_index+1}_1.png')
        stitch_images(processed_images[2], processed_images[3], f'./output/output_{batch_index+1}_2.png')
        stitch_images(f'./output/output_{batch_index+1}_1.png', f'./output/output_{batch_index+1}_2.png', f'./output/output_{batch_index+1}_final.png')

# 示例使用
camera_matrix = np.array([[800, 0, 640], [0, 800, 360], [0, 0, 1]], dtype=np.float32)
dist_coeffs_list = [
    np.array([-0.401, 0.59999991, 0, 0], dtype=np.float32),
    np.array([-0.3, 0.2, 0, 0], dtype=np.float32),
    np.array([-0.0733, 0.0833, 0, 0], dtype=np.float32),
    np.array([0.03, 0.00, 0, 0], dtype=np.float32)
]
rotation_angles = [-3.5865, 0, 0, 5]

# 图片列表的列表
image_batches = [
    ['../data/video1_frame1.jpg', '../data/video2_frame1.jpg', '../data/video3_frame1.jpg', '../data/video4_frame1.jpg'],
    # 添加更多批次的图像路径列表
]

process_image_batches(image_batches, camera_matrix, dist_coeffs_list, rotation_angles)

Processing batch 1
Saved undistorted and rotated image: ./output/undistorted_image_1.png
Undistort and rotate time: 0.01 seconds
Saved undistorted and rotated image: ./output/undistorted_image_2.png
Undistort and rotate time: 0.01 seconds
Saved undistorted and rotated image: ./output/undistorted_image_3.png
Undistort and rotate time: 0.01 seconds
Saved undistorted and rotated image: ./output/undistorted_image_4.png
Undistort and rotate time: 0.01 seconds
Stitched images: ./output/undistorted_image_1.png, ./output/undistorted_image_2.png to ./output/output_1_1.png
Stitching time: 0.25 seconds
Stitched images: ./output/undistorted_image_3.png, ./output/undistorted_image_4.png to ./output/output_1_2.png
Stitching time: 0.20 seconds
Stitched images: ./output/output_1_1.png, ./output/output_1_2.png to ./output/output_1_final.png
Stitching time: 0.28 seconds
